In [1]:
# 安装必要的库
!pip install -U openai scikit-learn tqdm pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 110.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 106.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigfr

In [2]:

import time
import json
import re
import os
import numpy as np
import pandas as pd

from openai import OpenAI
from kaggle_secrets import UserSecretsClient
from datasets import load_dataset
# 修改点：使用 notebook 专用进度条，显示更稳定
from tqdm.notebook import tqdm 
from sklearn.metrics import accuracy_score, f1_score

user_secrets = UserSecretsClient()
SILICONFLOW_API_KEY = user_secrets.get_secret("SILICONFLOW_API_KEY")

client = OpenAI(
    api_key=SILICONFLOW_API_KEY,
    base_url="https://api.siliconflow.cn/v1"
)

MODEL_NAME = "deepseek-ai/DeepSeek-V3.2"

In [3]:
import os
import pandas as pd

# --- 1. 核心路径配置 ---
DATA_ROOT = "/kaggle/input/text-cls/data"

# --- 2. 增强版数据映射 (指定确切文件名) ---
DATA_MAPPINGS = {
    "us_economy": {
        "folder": "01-nyt-sentiment",
        # 截图显示确切文件名为 3SU.csv
        "filename": "3SU.csv",
        "text_col": "text",   # 稍后会自动检测，如果不叫 text 会尝试智能匹配
        "label_col": "label", # 3SU 数据集通常叫 label 或 sentiment
        "id2label": {0: "Negative Sentiment", 1: "Positive Sentiment"}
    },
    "kavanaugh": {
        "folder": "02-twitter-stance",
        # 截图显示确切文件名为 kavanaugh_tweets_groundtruth.csv
        "filename": "kavanaugh_tweets_groundtruth.csv",
        "text_col": "text",   # 论文提到列名是 text
        "label_col": "stance", # 论文提到列名是 stance
        "id2label": {0: "negative attitudinal stance towards", 1: "positive attitudinal stance towards"}
    },
    # 针对 03 和 04 文件夹，截图中显示只有 all-x 和 all-y，需要合并
    "german_anger": {
        "folder": "03-emotion-angry",
        "filename": "all-x-translated.csv", # 优先读取翻译后的文本
        "label_filename": "all-y.csv",      # 标签在另一个文件
        "text_col": "text",
        "label_col": "anger_label",
        "id2label": {0: "Non-Angry", 1: "Angry"}
    },
    "eu_position": {
        "folder": "04-brexit-stance",
        "filename": "all-x.csv",       # 文本
        "label_filename": "all-y.csv", # 标签
        "text_col": "text",
        "label_col": "position",
        "id2label": {0: "Neutral towards Leave demands", 1: "Pro-Leave demands", 2: "Very Pro-Leave demands"}
    }
}

# --- 2. 原论文中的 Prompt ---
PROMPTS_ORIGINAL = {
    "us_economy": {
        "instruction": "You have been assigned the task of zero-shot text classification for sentiment analysis. Your objective is to classify a given text snippet into one of several possible class labels, based on the sentiment expressed in the text. Your output should consist of a single class label that best matches the sentiment expressed in the text. Your output should consist of a single class label that best matches the given text. Choose ONLY from the given class labels below and ONLY output the label without any other characters.",
        "labels": ["Negative Sentiment", "Positive Sentiment"]
    },
    "kavanaugh": {
        "instruction": "You have been assigned the task of zero-shot text classification for stance classification. Your objective is to classify a given text snippet into one of several possible class labels, based on the attitudinal stance towards the given text. Your output should consist of a single class label that best matches the stance expressed in the text. Your output should consist of a single class label that best matches the given text. Choose ONLY from the given class labels below and ONLY output the label without any other characters.",
        "labels": ["negative attitudinal stance towards", "positive attitudinal stance towards"]
    },
    "german_anger": {
        "instruction": "You have been assigned the task of zero-shot text classification for emotion classification. Your objective is to classify a given text snippet into one of several possible class labels, based on the anger level in the given text. Your output should consist of a single class label that best matches the anger expressed in the text. Choose ONLY from the given class labels below and ONLY output the label without any other characters.",
        "labels": ["Angry", "Non-Angry"]
    },
    "eu_position": {
        "instruction": "You have been assigned the task of zero-shot text classification for political texts on attitudinal stance towards Brexit and leave demands related to the European Union (EU). Your objective is to classify a given text snippet into one of several possible class labels, based on the stance towards Brexit and general leave demands in the given text. Your output should consist of a single class label that best matches the content expressed in the text. Choose ONLY from the given class labels below and ONLY output the label without any other characters.",
        "labels": ["Neutral towards Leave demands", "Pro-Leave demands", "Very Pro-Leave demands"]
    }
}

# --- 3. 优化后的 Prompt ---
PROMPTS_OPTIMIZED = {
    "us_economy": {
        "instruction": """### Role
You are an expert financial analyst.

### Task
Classify the sentiment of the following news snippet regarding the US Economy.

### Constraints
- Output exactly ONE label from the list below.
- Do NOT output explanations or punctuation.""",
        "labels": ["Negative Sentiment", "Positive Sentiment"]
    },
    "kavanaugh": {
        "instruction": """### Role
You are a political stance detector specializing in social media text.

### Task
Determine if the author of the tweet supports or opposes the nomination of Brett Kavanaugh.

### Constraints
- Output exactly ONE label from the list below.
- Do NOT output explanations.""",
        "labels": ["negative attitudinal stance towards", "positive attitudinal stance towards"]
    },
    "german_anger": {
        "instruction": """### Role
You are a linguistic expert specializing in emotion detection in political discourse.

### Task
Analyze the text and determine if it expresses 'Anger'.

### Constraints
- Output exactly ONE label from the list below.
- Do NOT output explanations.""",
        "labels": ["Angry", "Non-Angry"]
    },
    "eu_position": {
        "instruction": """### Role
You are a political scientist specializing in EU affairs and Brexit.

### Task
Classify the party's position regarding Brexit and EU leave demands.

### Constraints
- Output exactly ONE label from the list below.
- Do NOT output explanations.""",
        "labels": ["Neutral towards Leave demands", "Pro-Leave demands", "Very Pro-Leave demands"]
    }
}

# --- 4. 智能数据加载函数 (核心修复) ---
def load_dataset_smart(task_key):
    config = DATA_MAPPINGS[task_key]
    folder_path = os.path.join(DATA_ROOT, config['folder'])
    
    # 情况 A: 存在 label_filename，说明是分离的数据 (03, 04 文件夹)
    if "label_filename" in config:
        print(f"📂 Detected split files for {task_key}. Merging...")
        try:
            # 读取文本 (all-x)
            path_x = os.path.join(folder_path, config['filename'])
            df_x = pd.read_csv(path_x, encoding='latin1')
            # 如果没有表头，强制设为 text
            if config['text_col'] not in df_x.columns:
                df_x = pd.read_csv(path_x, encoding='latin1', header=None, names=[config['text_col']])
            
            # 读取标签 (all-y)
            path_y = os.path.join(folder_path, config['label_filename'])
            df_y = pd.read_csv(path_y, encoding='latin1')
            # 如果没有表头，强制设为 label_col
            if config['label_col'] not in df_y.columns:
                 df_y = pd.read_csv(path_y, encoding='latin1', header=None, names=[config['label_col']])
            
            # 合并
            df = pd.concat([df_x, df_y], axis=1)
            return df
        except Exception as e:
            print(f"❌ Merge failed: {e}")
            return None

    # 情况 B: 单个文件 (01, 02 文件夹)
    else:
        file_path = os.path.join(folder_path, config['filename'])
        print(f"📄 Loading specific file: {config['filename']}")
        try:
            # 尝试不同的编码
            try:
                df = pd.read_csv(file_path)
            except:
                df = pd.read_csv(file_path, encoding='latin1')
            
            # 自动修正列名 (如果 csv 里叫 'tweet_text' 但配置叫 'text')
            # 简单的列名规范化
            df.columns = [c.lower() for c in df.columns]
            
            # 检查关键列是否存在，如果不存在尝试模糊匹配
            if config['text_col'] not in df.columns:
                # 找一个看起来像文本的列
                candidates = ['sentence', 'tweet_text', 'tweet', 'content', 'text']
                for cand in candidates:
                    if cand in df.columns:
                        print(f"⚠️ Renaming column '{cand}' to '{config['text_col']}'")
                        df = df.rename(columns={cand: config['text_col']})
                        break
            
            return df
            
        except FileNotFoundError:
            print(f"❌ File not found: {file_path}")
            return None


In [4]:
from sklearn.metrics import classification_report

def run_comparison(task_key, prompt_type="original", sample_size=None):
    # 1. 加载数据
    df = load_dataset_smart(task_key)
    
    if df is None:
        print(f"❌ Failed to load data for {task_key}")
        return None
        
    data_config = DATA_MAPPINGS[task_key]
    
    # 检查文本列
    if data_config['text_col'] not in df.columns:
        print(f"❌ Column '{data_config['text_col']}' missing. Found: {df.columns.tolist()}")
        return None
        
    # 自动检查标签列
    label_col = data_config['label_col']
    if label_col not in df.columns:
        print(f"❌ Label column '{label_col}' not found. Found: {df.columns.tolist()}")
        return None

    if sample_size:
        df = df.head(sample_size)

    # 2. 选择 Prompt
    if prompt_type == "original":
        prompt_config = PROMPTS_ORIGINAL[task_key] 
    else:
        prompt_config = PROMPTS_OPTIMIZED[task_key]

    print(f"🚀 Running {task_key} | Mode: {prompt_type.upper()} | Samples: {len(df)}")
    
    results_records = []
    true_labels = []
    pred_labels = []
    
    pbar = tqdm(df.iterrows(), total=len(df), desc="Processing", ncols=800)
    
    for idx, row in pbar:
        text = str(row[data_config['text_col']])
        
        # --- 【关键修复】适配字符串标签 ---
        try:
            raw_label = row[label_col]
            
            # 如果是 NaN (空值)，跳过
            if pd.isna(raw_label):
                continue

            # 转成字符串并小写，去匹配配置里的键 (例如 "positive")
            # 这样 'mixed' 因为不在 id2label 里，就会被自动跳过，符合论文过滤逻辑
            label_key = str(raw_label).lower().strip()
            
            if label_key in data_config['id2label']:
                true_text = data_config['id2label'][label_key]
            else:
                # 尝试看看是不是数字 (兼容其他数据集)
                try:
                    int_label = int(raw_label)
                    if int_label in data_config['id2label']:
                        true_text = data_config['id2label'][int_label]
                    else:
                        continue # 既不是已知字符串，也不是已知数字，跳过
                except:
                    continue # 无法转换，跳过
                    
        except Exception as e:
            continue 
            
        # 构建 Prompt
        labels_str = ", ".join([f"'{l}'" for l in prompt_config['labels']])
        
        if prompt_type == "original":
            final_prompt = f"{prompt_config['instruction']}\n\nText: {text}\nLabels: {labels_str}\nAnswer:"
        else:
            final_prompt = f"{prompt_config['instruction']}\n\n### Labels\n{labels_str}\n\n### Input Text\n{text}\n\n### Answer"

        # 调用 LLM
        raw_pred = query_llm(final_prompt)
        pred_clean = normalize_prediction(raw_pred, prompt_config['labels'])
        
        # 记录
        results_records.append({
            "text": text,
            "true_label": true_text,
            "pred_label": pred_clean,
            "is_correct": true_text == pred_clean,
            "raw_response": raw_pred 
        })
        
        true_labels.append(true_text)
        pred_labels.append(pred_clean)
        
        pbar.set_postfix({"Last": pred_clean[:10]})
        time.sleep(0.3)

    if not true_labels:
        print("⚠️ No valid labels found. Please check your data mappings.")
        return None

    # 计算指标
    print(f"\n📊 Detailed Classification Report ({prompt_type.upper()}):")
    print(classification_report(true_labels, pred_labels, zero_division=0))

    result_filename = f"prediction_{task_key}_{prompt_type}.csv"
    pd.DataFrame(results_records).to_csv(result_filename, index=False, encoding='utf-8-sig')
    print(f"💾 Saved to: {result_filename}")

    acc = accuracy_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels, average='macro')
    
    return {"accuracy": acc, "f1": f1}

In [5]:
import time

def find_csv_in_folder(folder_path):
    """在指定文件夹中找到第一个CSV文件"""
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Directory not found: {folder_path}")
    
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            return os.path.join(folder_path, file)
    raise FileNotFoundError(f"No CSV file found in {folder_path}")

def query_llm(prompt):
    """
    调用 LLM，包含自动重试 (限速/系统繁忙) 和 自动跳过 (敏感内容)
    """
    # 检查 Key 是否存在
    if not SILICONFLOW_API_KEY or "sk-" not in SILICONFLOW_API_KEY:
        tqdm.write("❌ Critical: API Key is missing or invalid!")
        return None

    max_retries = 10 # 最大重试次数
    base_wait_time = 3 # 基础等待时间

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1, 
            )
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            error_msg = str(e).lower()
            
            # --- 情况 1: 敏感内容拦截 (400) ---
            if "400" in error_msg and ("input data" in error_msg or "sensitive" in error_msg):
                # tqdm.write(f"⚠️ 跳过敏感内容") # 可选：打印日志
                return None 

            # --- 情况 2: 限速 (429) 或 服务端繁忙 (500/502/503/busy) ---
            elif ("rpm" in error_msg or "429" in error_msg or "limit" in error_msg or 
                  "503" in error_msg or "502" in error_msg or "500" in error_msg or "busy" in error_msg):
                
                wait_time = base_wait_time * (attempt + 1)
                # 打印提示
                if "503" in error_msg or "busy" in error_msg:
                    tqdm.write(f"⏳ 服务器繁忙 (503 Busy)，等待 {wait_time}s 后重试... ({attempt+1}/{max_retries})")
                else:
                    tqdm.write(f"⏳ 限速中，等待 {wait_time}s 后重试...")
                
                time.sleep(wait_time)
                
            # --- 情况 3: 其他错误 (如认证失败) ---
            else:
                tqdm.write(f"⚠️ API Error: {error_msg[:100]}...")
                return None
    
    tqdm.write("❌ 重试多次失败，放弃该样本。")
    return None

def normalize_prediction(pred_raw, valid_labels):
    if not pred_raw:
        return "Unknown"
    
    pred_lower = pred_raw.lower().strip()
    pred_lower = pred_lower.rstrip(".,\"'")
    
    for label in valid_labels:
        if label.lower() in pred_lower:
            return label
            
    return "Unknown"

In [6]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

def run_comparison(task_key, prompt_type="original", sample_size=None, start_idx=0):
    # 1. 加载数据
    df = load_dataset_smart(task_key)
    
    if df is None:
        print(f"❌ Failed to load data for {task_key}")
        return None
        
    data_config = DATA_MAPPINGS[task_key]
    
    # 检查文本列
    if data_config['text_col'] not in df.columns:
        print(f"❌ Column '{data_config['text_col']}' missing. Found: {df.columns.tolist()}")
        return None
        
    # 自动检查标签列
    label_col = data_config['label_col']
    if label_col not in df.columns:
        print(f"❌ Label column '{label_col}' not found. Found: {df.columns.tolist()}")
        return None

    if sample_size:
        df = df.head(sample_size)

    total_samples = len(df)

    # 2. 选择 Prompt
    if prompt_type == "original":
        prompt_config = PROMPTS_ORIGINAL[task_key] 
    else:
        prompt_config = PROMPTS_OPTIMIZED[task_key]

    print(
        f"🚀 Running {task_key} | Mode: {prompt_type.upper()} | "
        f"Samples: {total_samples} | Start from index: {start_idx}"
    )
    
    results_records = []
    true_labels = []
    pred_labels = []
    
    pbar = tqdm(
        df.iterrows(),
        total=total_samples,
        desc="Processing",
        ncols=800,
        initial=start_idx
    )
    
    for idx, row in pbar:
        # ✅ 断点续跑：跳过已完成样本
        if idx < start_idx:
            continue

        text = str(row[data_config['text_col']])
        
        # --- 适配字符串 / 数字标签 ---
        try:
            raw_label = row[label_col]
            
            if pd.isna(raw_label):
                continue

            label_key = str(raw_label).lower().strip()
            
            if label_key in data_config['id2label']:
                true_text = data_config['id2label'][label_key]
            else:
                try:
                    int_label = int(raw_label)
                    if int_label in data_config['id2label']:
                        true_text = data_config['id2label'][int_label]
                    else:
                        continue
                except:
                    continue
                    
        except Exception:
            continue 
            
        # 构建 Prompt
        labels_str = ", ".join([f"'{l}'" for l in prompt_config['labels']])
        
        if prompt_type == "original":
            final_prompt = (
                f"{prompt_config['instruction']}\n\n"
                f"Text: {text}\n"
                f"Labels: {labels_str}\n"
                f"Answer:"
            )
        else:
            final_prompt = (
                f"{prompt_config['instruction']}\n\n"
                f"### Labels\n{labels_str}\n\n"
                f"### Input Text\n{text}\n\n"
                f"### Answer"
            )

        # 调用 LLM
        raw_pred = query_llm(final_prompt)
        pred_clean = normalize_prediction(raw_pred, prompt_config['labels'])
        
        # 记录
        results_records.append({
            "index": idx,
            "text": text,
            "true_label": true_text,
            "pred_label": pred_clean,
            "is_correct": true_text == pred_clean,
            "raw_response": raw_pred 
        })
        
        true_labels.append(true_text)
        pred_labels.append(pred_clean)
        
        pbar.set_postfix({"Last": pred_clean[:10]})
        time.sleep(0.3)

    if not true_labels:
        print("⚠️ No valid labels found. Please check your data mappings.")
        return None

    # 计算指标
    print(f"\n📊 Detailed Classification Report ({prompt_type.upper()}):")
    print(classification_report(true_labels, pred_labels, zero_division=0))

    result_filename = f"prediction_{task_key}_{prompt_type}.csv"
    pd.DataFrame(results_records).to_csv(
        result_filename,
        index=False,
        encoding='utf-8-sig'
    )
    print(f"💾 Saved to: {result_filename}")

    acc = accuracy_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels, average='macro')
    
    return {"accuracy": acc, "f1": f1}


In [7]:
# 选择一个任务进行对比，例如 Kavanaugh 推特立场分析
TASK_NAME = "eu_position" 
SAMPLE_N = None  # 建议先用 20-50 个样本快速测试，确认无误后再跑全量

print("--- 1. Running Baseline (Original Paper Prompts) ---")
res_orig = run_comparison(TASK_NAME, prompt_type="original", sample_size=SAMPLE_N)

print("\n--- 2. Running Experiment (Optimized Prompts) ---")
res_opt = run_comparison(TASK_NAME, prompt_type="optimized", sample_size=SAMPLE_N)

# 输出对比
print(f"\n🏆 === Comparison Result: {TASK_NAME} ===")
print(f"Original  -> Acc: {res_orig['accuracy']:.4f}, F1: {res_orig['f1']:.4f}")
print(f"Optimized -> Acc: {res_opt['accuracy']:.4f}, F1: {res_opt['f1']:.4f}")

diff_acc = res_opt['accuracy'] - res_orig['accuracy']
print(f"Improvement: {diff_acc:+.4f}")

--- 1. Running Baseline (Original Paper Prompts) ---
📂 Detected split files for eu_position. Merging...
🚀 Running eu_position | Mode: ORIGINAL | Samples: 3349 | Start from index: 0


Processing:   0%|                                                                                             …


📊 Detailed Classification Report (ORIGINAL):
                               precision    recall  f1-score   support

Neutral towards Leave demands       0.12      0.86      0.22       292
            Pro-Leave demands       0.84      0.40      0.54      2764
       Very Pro-Leave demands       0.00      0.00      0.00       293

                     accuracy                           0.41      3349
                    macro avg       0.32      0.42      0.25      3349
                 weighted avg       0.70      0.41      0.47      3349

💾 Saved to: prediction_eu_position_original.csv

--- 2. Running Experiment (Optimized Prompts) ---
📂 Detected split files for eu_position. Merging...
🚀 Running eu_position | Mode: OPTIMIZED | Samples: 3349 | Start from index: 0


Processing:   0%|                                                                                             …


📊 Detailed Classification Report (OPTIMIZED):
                               precision    recall  f1-score   support

Neutral towards Leave demands       0.13      0.78      0.23       292
            Pro-Leave demands       0.85      0.49      0.62      2764
       Very Pro-Leave demands       0.00      0.00      0.00       293

                     accuracy                           0.48      3349
                    macro avg       0.33      0.43      0.28      3349
                 weighted avg       0.71      0.48      0.53      3349

💾 Saved to: prediction_eu_position_optimized.csv

🏆 === Comparison Result: eu_position ===
Original  -> Acc: 0.4055, F1: 0.2530
Optimized -> Acc: 0.4763, F1: 0.2831
Improvement: +0.0708
